# basic rex tutorial (using local hsds server)
* use conda env: rex 
* make sure you start hsds in terminal:
```
conda activate rex
hsds
```
* for more info on rex see docs at:
* https://nrel.github.io/rex/


In [37]:
import pandas as pd
import os

import math
import numpy as np


# show multiple cell outputs
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 100)

# cufflings for easy pandas df plotting (optional)
import cufflinks as cl

cl.go_offline()

import matplotlib.pyplot as plt

from rex import WindX
from rex import Resource
import h5pyd
import rex
from rex import MultiFileResourceX, ResourceX, MultiTimeResourceX


### too see all AWS NREL .h5 datasets available through HSDS
* use "hsls /nrel/" command in your terminal
* or use h5pyd cell below


### first retrieve wind data at one lat/lon

In [3]:
### WTK wind hourly:
# /nrel/wtk/conus/wtk_conus_2007.h5 ... wtk_conus_2014.h5 and wtk_conus_2018.h5 ... wtk_conus_2020.h5
### WTK wind 5-min:
# /nrel/wtk/conus-5min/wtk_conus_2007.h5 ... wtk_conus_2014.h5


## select resource file
res_file = "/nrel/wtk/conus-5min/wtk_conus_2014.h5"
res_file = "/nrel/wtk/conus/wtk_conus_2014.h5"

## select location
lat_lon = (39.913561, -105.222422)  ## Flatirons wind site
lat_lon = (40.015587, -105.279457)  ## in Boulder Canyon and Broadway

with WindX(res_file, hsds=True) as f:
    df = f.get_lat_lon_df("windspeed_100m", lat_lon)

In [4]:
df

,851061
time_index,
2014-01-01 00:00:00+00:00,3.38
2014-01-01 01:00:00+00:00,2.68
2014-01-01 02:00:00+00:00,1.51
2014-01-01 03:00:00+00:00,1.49
2014-01-01 04:00:00+00:00,4.70
...,...
2014-12-31 19:00:00+00:00,0.76
2014-12-31 20:00:00+00:00,1.12
2014-12-31 21:00:00+00:00,1.93


In [5]:
# select timeslice and plot
start = pd.Timestamp("2014-07-22 00:00:00", tz="utc")
end = pd.Timestamp("2014-07-25 00:00:00", tz="utc")

df.loc[start:end].iplot()

In [6]:
### save as .csv
filename = "boulder_100m_1h_wind_2014.csv"
df.to_csv(filename)

In [7]:
### read saved .csv
filename = "boulder_100m_1h_wind_2014.csv"
df = pd.read_csv(filename, parse_dates=True, index_col=0)
df

,851061
time_index,
2014-01-01 00:00:00+00:00,3.38
2014-01-01 01:00:00+00:00,2.68
2014-01-01 02:00:00+00:00,1.51
2014-01-01 03:00:00+00:00,1.49
2014-01-01 04:00:00+00:00,4.70
...,...
2014-12-31 19:00:00+00:00,0.76
2014-12-31 20:00:00+00:00,1.12
2014-12-31 21:00:00+00:00,1.93


### browse AWS NREL datasets via hsds terminal commands
* must have hsds server running
* more commands at: https://github.com/HDFGroup/hsds_examples


In [57]:
### check if hsds server is running
!hsinfo


server name: Highly Scalable Data Service (HSDS)
server state: READY
endpoint: http://localhost:5101
username: anonymous 
password: 
server version: 0.8.4
node count: 4
up: 9 hours 55 min 8 sec
h5pyd version: 0.18.0


In [58]:
### browse AWS NREL dir tree
!hsls /nrel/

nrel_admin                                          folder   2017-08-20 15:46:58 /nrel/
nrel_admin                                          folder   2020-08-12 14:31:43 /nrel/US_wave
nrel_admin                                          folder   2021-02-01 15:42:38 /nrel/building_synthetic_dataset
nrel_admin                                          folder   2021-09-23 15:11:05 /nrel/dsgrid-2018-efs
nrel_admin                                          folder   2024-09-11 06:09:15 /nrel/ncdb
nrel_admin                                          folder   2019-10-01 08:08:23 /nrel/nsrdb
nrel_admin                                          folder   2020-09-03 08:51:36 /nrel/porotomo
nrel_admin                                          folder   2023-04-20 16:56:35 /nrel/sup3rcc
nrel_admin                                          folder   2024-09-19 13:29:44 /nrel/test
nrel_admin                                          folder   2020-08-31 11:36:19 /nrel/umcm
nrel_admin                              

In [60]:
### sup3rcc folder
!hsls /nrel/sup3rcc/

nrel_admin                                          folder   2023-04-20 16:56:35 /nrel/sup3rcc/
nrel_admin                                          folder   2023-09-05 13:13:38 /nrel/sup3rcc/conus_ecearth3_ssp585_r1i1p1f1
nrel_admin                                          folder   2023-04-20 16:57:53 /nrel/sup3rcc/conus_mriesm20_ssp585_r1i1p1f1
3 items


In [62]:
### get info of one resource file (domain)
!hsstat /nrel/sup3rcc/conus_ecearth3_ssp585_r1i1p1f1/sup3rcc_conus_ecearth3_ssp585_r1i1p1f1_2053.h5

domain: /nrel/sup3rcc/conus_ecearth3_ssp585_r1i1p1f1/sup3rcc_conus_ecearth3_ssp585_r1i1p1f1_2053.h5
    owner:           nrel_admin
    id:              g-805bcf37-1e7135f2-08d3-47bf96-f9bd7a
    last modified:   2024-02-28 15:06:30
    last scan:       2024-02-28 14:40:48
    md5 sum:         14abbb672b058cc1f828723ea30a4cb7
    total_size:      552208282025
    allocated_bytes: 3312000
    metadata_bytes:  47676
    linked_bytes:    552204921848
    num objects:     27
    num chunks:      12
    linked chunks:   276065


### browse AWS NREL datasets via h5pyd
* uncomment/modify blocks as needed



In [8]:
### to see AWS NREL data directories
folder = "/nrel/"
with h5pyd.Folder(folder) as f:
    print(f"\n {folder}:")
    for i in range(len(list(f))):
        print(list(f)[i])


folder = "/nrel/sup3rcc/"
with h5pyd.Folder(folder) as f:
    print(f"\n {folder}:")
    for i in range(len(list(f))):
        print(list(f)[i])

# folder = "/nrel/sup3rcc/conus_ecearth3_ssp585_r1i1p1f1/"
# with h5pyd.Folder(folder) as f:
#     print(f"\n {folder}:")
#     for i in range(len(list(f))):
#         print(list(f)[i])


 /nrel/:
US_wave
building_synthetic_dataset
dsgrid-2018-efs
ncdb
nsrdb
porotomo
sup3rcc
test
umcm
wtk-led
wtk-us.h5
wtk

 /nrel/sup3rcc/:
conus_ecearth3_ssp585_r1i1p1f1
conus_mriesm20_ssp585_r1i1p1f1


### retrieve resource by filename (domain)
* see what variables are avialable in res_file (also get meta data, time index, and coords)

In [73]:
### select resource filename
filename = "/nrel/sup3rcc/conus_ecearth3_ssp585_r1i1p1f1/sup3rcc_conus_ecearth3_ssp585_r1i1p1f1_2050.h5"
### resource
res = ResourceX(filename, hsds=True)

meta_df = res.meta
ti = res.time_index
coords = res.coordinates

print("meta_df:")
meta_df
print("time index:")
ti
print("coords:")
coords
print("dsets:")
res.dsets

meta_df:


,latitude,longitude,timezone,elevation,country,state,county,offshore,eez
gid,,,,,,,,,
0,51.213902,-129.009369,-9.0,0,Canada,nan,nan,1,1
1,51.213902,-128.981247,-9.0,0,Canada,nan,nan,1,1
2,51.213902,-128.953125,-9.0,0,Canada,nan,nan,1,1
3,51.213902,-128.925003,-9.0,0,Canada,nan,nan,1,1
4,51.213902,-128.896881,-9.0,0,Canada,nan,nan,1,1
...,...,...,...,...,...,...,...,...,...
2299995,23.171883,-64.462502,-4.0,0,nan,nan,nan,1,0
2299996,23.171883,-64.434372,-4.0,0,nan,nan,nan,1,0
2299997,23.171883,-64.406250,-4.0,0,nan,nan,nan,1,0


time index:


DatetimeIndex(['2050-01-01 00:00:00+00:00', '2050-01-01 01:00:00+00:00',
               '2050-01-01 02:00:00+00:00', '2050-01-01 03:00:00+00:00',
               '2050-01-01 04:00:00+00:00', '2050-01-01 05:00:00+00:00',
               '2050-01-01 06:00:00+00:00', '2050-01-01 07:00:00+00:00',
               '2050-01-01 08:00:00+00:00', '2050-01-01 09:00:00+00:00',
               ...
               '2050-12-31 14:00:00+00:00', '2050-12-31 15:00:00+00:00',
               '2050-12-31 16:00:00+00:00', '2050-12-31 17:00:00+00:00',
               '2050-12-31 18:00:00+00:00', '2050-12-31 19:00:00+00:00',
               '2050-12-31 20:00:00+00:00', '2050-12-31 21:00:00+00:00',
               '2050-12-31 22:00:00+00:00', '2050-12-31 23:00:00+00:00'],
              dtype='datetime64[ns, UTC]', length=8760, freq=None)

coords:


array([[  51.2139  , -129.00937 ],
       [  51.2139  , -128.98125 ],
       [  51.2139  , -128.95312 ],
       ...,
       [  23.171883,  -64.40625 ],
       [  23.171883,  -64.37813 ],
       [  23.171883,  -64.35    ]], dtype=float32)

dsets:


['dhi',
 'dni',
 'ghi',
 'meta',
 'pressure_0m',
 'relativehumidity_2m',
 'temperature_2m',
 'time_index',
 'winddirection_100m',
 'winddirection_10m',
 'winddirection_200m',
 'windspeed_100m',
 'windspeed_10m',
 'windspeed_200m']

### accessing raw data by gids from res

In [74]:
### select variable
var = "windspeed_100m"

### time index
ti = res.time_index
### select gid (grid points) from index of meta_df (above)
gids = [0, 1, 2, 3, 4]

df = pd.DataFrame(
    res[var, :, gids],
    index=res.time_index,
)

df
df.iloc[1:100].iplot()

,0,1,2,3,4
2050-01-01 00:00:00+00:00,9.64,9.53,9.51,9.63,9.66
2050-01-01 01:00:00+00:00,10.33,10.25,10.30,10.46,10.54
2050-01-01 02:00:00+00:00,10.53,10.70,10.78,10.77,10.63
2050-01-01 03:00:00+00:00,10.80,11.03,11.02,10.88,10.78
2050-01-01 04:00:00+00:00,10.74,10.71,10.66,10.68,10.68
...,...,...,...,...,...
2050-12-31 19:00:00+00:00,9.04,9.11,9.19,9.35,9.49
2050-12-31 20:00:00+00:00,9.37,9.28,9.27,9.47,9.53
2050-12-31 21:00:00+00:00,9.25,9.20,9.14,9.21,9.30
2050-12-31 22:00:00+00:00,9.48,9.48,9.32,9.16,9.21


### accessing data by box of gids
* large box might take a while

In [75]:
### select corners of the box (lat/lon)

# ### entire state of CO (~1GB)
# top = 40.931933
# left = -108.997954
# bottom = 37.102284
# right = -102.393198

### box over CO skiing areas (~40MB)
top = 39.336103
left = -107.208223
bottom = 38.545948
right = -105.939496

gids = res.box_gids((top, left), (bottom, right))

### choose a variable for a year (see dsets above)
### to select another year reload res using another .h5 file

# var = "temperature_2m"
var = "windspeed_100m"

ti = res.time_index

df = pd.DataFrame(res[var, :, gids], index=res.time_index, columns=gids)
df.index.name = "timestamp"
df.columns.name = "gid"


print(f"df for variable {var}:")
df
df.info()

### select meta data only for needed gids
meta_df = meta_df.loc[df.columns]
print("meta_df:")
meta_df
meta_df.info()

df for variable windspeed_100m:


gid,975976,975977,975978,975979,975980,975981,975982,975983,975984,975985,975986,975987,975988,975989,975990,975991,975992,975993,975994,975995,975996,975997,975998,975999,976000,976001,976002,976003,976004,976005,976006,976007,976008,976009,976010,976011,976012,976013,976014,976015,976016,976017,976018,976019,976020,978276,978277,978278,978279,978280,...,1035816,1035817,1035818,1035819,1035820,1038076,1038077,1038078,1038079,1038080,1038081,1038082,1038083,1038084,1038085,1038086,1038087,1038088,1038089,1038090,1038091,1038092,1038093,1038094,1038095,1038096,1038097,1038098,1038099,1038100,1038101,1038102,1038103,1038104,1038105,1038106,1038107,1038108,1038109,1038110,1038111,1038112,1038113,1038114,1038115,1038116,1038117,1038118,1038119,1038120
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2050-01-01 00:00:00+00:00,4.09,2.58,2.52,2.71,0.68,3.79,5.14,2.21,3.47,5.63,7.01,11.53,14.16,10.05,4.88,3.26,2.25,2.37,4.36,5.87,6.59,4.37,3.38,10.66,12.97,13.04,12.88,13.17,10.410000,8.83,10.49,11.08,9.16,6.94,7.10,7.52,7.46,7.080000,6.53,9.95,11.73,11.33,9.460000,7.78,8.65,5.00,2.29,2.21,1.93,2.89,...,4.86,5.47,3.91,2.96,4.37,6.29,5.56,5.76,4.76,4.13,5.29,4.62,2.54,2.76,5.67,6.92,9.09,7.97,6.05,5.54,7.72,9.53,7.29,4.47,4.51,5.33,5.55,6.78,7.52,7.98,7.73,6.84,6.22,6.38,4.91,2.24,0.60,1.98,3.62,4.12,3.00,2.42,3.88,7.27,4.03,2.99,3.15,2.87,2.82,3.53
2050-01-01 01:00:00+00:00,3.57,1.50,2.85,4.19,1.52,3.43,4.73,1.71,6.99,10.01,9.36,10.52,10.54,7.14,3.97,2.45,1.81,2.25,3.05,2.81,1.63,2.99,7.58,13.58,15.19,13.18,13.20,12.36,8.940000,7.96,11.30,10.94,8.04,6.59,8.27,9.14,9.82,7.090000,6.35,9.07,9.98,9.29,8.930000,8.06,7.11,4.14,2.11,1.98,2.76,3.05,...,1.22,2.47,3.97,4.68,4.83,6.63,5.42,6.63,4.96,4.39,7.39,7.20,4.08,4.40,6.42,5.90,9.00,8.59,5.91,5.18,7.50,7.11,4.92,4.74,3.64,3.07,4.32,6.02,6.17,5.96,3.77,3.90,4.82,5.39,4.15,3.75,2.32,1.77,1.14,1.75,1.04,3.33,2.27,4.24,3.00,0.97,0.87,2.00,4.64,5.17
2050-01-01 02:00:00+00:00,4.84,3.89,0.92,0.80,1.84,5.10,2.03,4.51,8.97,12.12,11.96,11.16,10.00,6.57,3.43,2.40,1.74,0.97,1.91,3.31,3.44,9.67,15.07,15.58,13.91,10.49,10.36,10.63,10.010000,8.93,10.71,10.71,6.29,6.53,7.31,7.56,10.23,6.830000,5.78,7.53,8.78,10.11,9.770000,9.33,7.91,5.29,3.56,1.24,1.57,1.57,...,1.31,3.55,5.82,5.10,4.05,8.16,5.15,6.30,5.51,4.83,6.77,5.95,3.95,3.97,4.36,6.55,7.84,5.76,4.79,4.92,7.36,4.85,3.49,2.62,3.47,5.18,7.43,6.16,4.60,4.99,4.14,4.15,3.60,3.23,2.27,2.44,1.68,1.93,1.74,0.77,1.89,2.69,3.33,2.87,1.29,2.07,2.21,4.74,7.82,8.90
2050-01-01 03:00:00+00:00,2.43,2.19,0.83,1.36,2.65,2.20,5.81,10.30,10.19,10.92,11.54,11.68,9.54,5.90,3.75,3.57,3.30,2.07,2.25,4.32,4.35,11.20,13.82,14.19,14.36,11.32,10.99,10.90,9.320000,7.73,11.26,12.21,6.41,5.22,5.38,7.32,11.06,8.560000,6.64,8.19,11.18,14.55,11.000000,9.02,8.02,3.69,2.73,0.08,2.37,1.87,...,4.51,8.56,9.23,6.62,6.20,8.10,6.15,5.26,3.99,5.50,9.74,8.24,6.69,7.38,4.19,8.17,7.86,4.03,2.24,3.63,4.63,3.36,2.87,4.51,7.20,7.66,8.39,6.66,3.83,5.36,5.81,3.91,3.62,3.14,1.56,1.92,1.15,1.81,1.74,2.06,2.14,2.71,3.89,3.58,2.19,0.92,5.20,9.48,11.07,10.37
2050-01-01 04:00:00+00:00,0.84,1.40,1.26,1.69,2.44,4.59,5.97,9.55,11.37,11.02,11.17,9.35,5.19,3.08,3.14,2.44,3.11,3.23,1.60,1.57,5.68,12.33,12.17,12.09,13.88,11.85,9.80,10.22,8.710000,8.94,12.66,11.85,8.13,5.41,4.66,8.22,13.03,6.860000,5.74,9.51,10.84,11.76,8.550000,4.40,4.30,1.11,1.02,0.59,2.86,2.67,...,4.84,10.35,9.38,7.68,7.73,6.75,5.51,6.06,4.30,4.26,8.41,7.94,6.56,6.21,2.76,5.55,8.08,5.26,4.25,5.85,6.17,4.34,3.03,3.58,5.87,6.46,6.89,5.86,4.12,4.96,6.28,4.79,2.42,1.90,1.56,0.44,0.73,2.29,3.08,1.97,1.01,1.70,2.35,2.63,1.31,1.73,5.94,9.80,12.31,11.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8760 entries, 2050-01-01 00:00:00+00:00 to 2050-12-31 23:00:00+00:00
Columns: 1260 entries, 975976 to 1038120
dtypes: float32(1260)
memory usage: 42.2 MB
meta_df:


,latitude,longitude,timezone,elevation,country,state,county,offshore,eez
gid,,,,,,,,,
975976,39.312191,-107.184372,-7.0,2344,United States,Colorado,Pitkin,0,1
975977,39.312191,-107.156250,-7.0,2507,United States,Colorado,Pitkin,0,1
975978,39.312191,-107.128128,-7.0,2474,United States,Colorado,Pitkin,0,1
975979,39.312191,-107.099998,-7.0,2481,United States,Colorado,Pitkin,0,1
975980,39.312191,-107.071877,-7.0,2440,United States,Colorado,Pitkin,0,1
...,...,...,...,...,...,...,...,...,...
1038116,38.554298,-106.059372,-7.0,2197,United States,Colorado,Chaffee,0,1
1038117,38.554298,-106.031250,-7.0,2188,United States,Colorado,Chaffee,0,1
1038118,38.554298,-106.003128,-7.0,2188,United States,Colorado,Chaffee,0,1


<class 'pandas.core.frame.DataFrame'>
Index: 1260 entries, 975976 to 1038120
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   latitude   1260 non-null   float32
 1   longitude  1260 non-null   float32
 2   timezone   1260 non-null   float32
 3   elevation  1260 non-null   int16  
 4   country    1260 non-null   object 
 5   state      1260 non-null   object 
 6   county     1260 non-null   object 
 7   offshore   1260 non-null   int16  
 8   eez        1260 non-null   int16  
dtypes: float32(3), int16(3), object(3)
memory usage: 61.5+ KB


### multi-year data retrieval
* for large geographic extent can run out of memory

In [77]:
### one variable, multiple years, multiple gids

### choose a variable (see dsets above)
var = "temperature_2m"
# var = "windspeed_10m"
# var = "windspeed_100m"

### chose Global Climate Model
gcm = "ecearth3"  # European GCM
# gcm = "mriesm20"  # Japanese GCM

data_dir = "/nrel/sup3rcc/conus_ecearth3_ssp585_r1i1p1f1/"
# sup3rcc_conus_ecearth3_ssp585_r1i1p1f1_2042.h5

### use wildcard * for multiple years
filenames = os.path.join(data_dir, f"sup3rcc_conus_{gcm}_ssp585_r1i1p1f1_201*.h5")

### create resource
res = MultiTimeResourceX(filenames, hsds=True)

### select geographic extent (gids)

### box over CO skiing areas (~40MB for one year)
top = 39.336103
left = -107.208223
bottom = 38.545948
right = -105.939496

gids = res.box_gids((top, left), (bottom, right))

ti = res.time_index

### make df
df = pd.DataFrame(res[var, :, gids], index=res.time_index, columns=gids)
df.index.name = "timestamp"
df.columns.name = "gid"

print(f"df for variable {var}:")
df
df.info()

### select meta data only for needed gids
meta_df = res.meta.loc[gids]
print("meta_df:")
meta_df
meta_df.info()

df for variable temperature_2m:


gid,975976,975977,975978,975979,975980,975981,975982,975983,975984,975985,975986,975987,975988,975989,975990,975991,975992,975993,975994,975995,975996,975997,975998,975999,976000,976001,976002,976003,976004,976005,976006,976007,976008,976009,976010,976011,976012,976013,976014,976015,976016,976017,976018,976019,976020,978276,978277,978278,978279,978280,...,1035816,1035817,1035818,1035819,1035820,1038076,1038077,1038078,1038079,1038080,1038081,1038082,1038083,1038084,1038085,1038086,1038087,1038088,1038089,1038090,1038091,1038092,1038093,1038094,1038095,1038096,1038097,1038098,1038099,1038100,1038101,1038102,1038103,1038104,1038105,1038106,1038107,1038108,1038109,1038110,1038111,1038112,1038113,1038114,1038115,1038116,1038117,1038118,1038119,1038120
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00+00:00,1.91,0.80,0.60,0.61,0.95,0.07,2.37,2.72,0.34,2.43,0.14,0.98,-2.12,-2.95,-3.89,-2.48,-1.88,-2.43,-1.98,0.16,-0.67,-2.13,-6.07,-4.17,-6.75,-5.73,-5.93,-6.27,-4.97,-2.41,-1.61,-2.52,-4.80,-4.11,-2.05,-5.15,-8.26,-8.34,-4.99,-4.69,-2.30,-4.39,-5.47,-3.62,-3.33,-0.66,-2.58,-2.45,-2.06,-0.68,...,2.28,2.20,1.18,0.27,-0.38,0.19,-0.36,0.70,1.81,1.40,2.09,2.27,2.28,3.13,3.34,2.96,2.36,2.32,1.80,1.71,0.87,0.87,0.68,1.11,1.63,0.98,-1.16,-1.89,-3.02,-1.07,-1.58,-3.58,-2.99,-3.15,-4.38,-5.43,-2.90,-1.83,-0.61,0.37,0.77,0.35,0.79,0.86,2.07,2.89,2.94,2.93,1.38,-0.25
2015-01-01 01:00:00+00:00,0.20,-1.20,-1.11,-1.20,-0.58,-1.52,0.55,0.95,-1.37,0.72,-1.20,-0.64,-3.72,-4.36,-5.41,-3.64,-3.46,-3.49,-3.47,-1.12,-2.27,-3.10,-7.70,-5.67,-8.28,-7.64,-7.49,-7.71,-6.64,-3.92,-3.05,-3.85,-6.15,-5.84,-4.23,-6.59,-9.57,-9.85,-6.82,-6.40,-4.11,-5.84,-6.79,-4.94,-4.65,-2.65,-4.72,-4.26,-3.73,-2.22,...,1.02,1.02,0.13,-1.03,-1.79,-0.84,-1.55,-0.45,0.67,0.18,0.99,1.02,1.04,1.72,2.14,1.69,1.32,1.14,0.37,0.27,-0.44,-0.63,-0.82,-0.35,0.15,-0.43,-2.89,-3.48,-4.97,-3.31,-2.86,-5.25,-4.75,-5.01,-5.87,-7.21,-4.94,-3.55,-2.40,-1.46,-1.06,-1.07,-0.75,-0.52,0.66,1.57,1.71,1.77,0.44,-1.47
2015-01-01 02:00:00+00:00,-2.09,-3.33,-3.16,-3.37,-2.98,-3.77,-1.85,-1.81,-3.64,-2.17,-3.29,-2.83,-6.07,-6.94,-7.64,-5.94,-5.98,-5.82,-5.60,-3.43,-4.65,-5.78,-9.84,-8.50,-10.86,-9.96,-10.07,-10.56,-9.17,-6.64,-5.52,-6.37,-8.62,-8.12,-6.56,-9.05,-11.97,-12.29,-9.40,-8.48,-6.71,-8.33,-9.15,-7.46,-6.97,-4.91,-6.88,-6.37,-5.98,-4.70,...,-0.09,-0.32,-1.10,-2.24,-3.00,-3.12,-3.72,-2.66,-1.77,-2.09,-1.12,-1.26,-1.09,-0.26,-0.01,-0.56,-0.85,-1.07,-1.76,-1.94,-2.46,-2.70,-2.89,-2.53,-1.96,-2.75,-5.03,-5.47,-6.88,-4.95,-5.14,-7.58,-6.96,-7.27,-8.31,-9.40,-6.98,-5.72,-4.41,-3.48,-3.05,-2.95,-2.35,-2.01,-0.76,0.18,0.47,0.48,-0.86,-2.60
2015-01-01 03:00:00+00:00,-3.27,-4.76,-4.40,-4.55,-4.13,-4.98,-3.23,-2.99,-4.75,-3.24,-4.44,-4.13,-7.12,-8.28,-9.14,-7.39,-7.16,-7.18,-7.12,-4.90,-6.10,-7.07,-11.01,-9.87,-11.87,-11.37,-11.36,-11.66,-10.59,-7.93,-6.92,-7.72,-9.95,-9.41,-7.92,-10.26,-13.42,-13.36,-10.44,-9.67,-7.88,-9.59,-10.29,-8.53,-8.16,-6.00,-8.17,-7.62,-7.28,-5.78,...,-0.98,-0.93,-1.84,-3.04,-3.99,-4.41,-5.03,-3.95,-2.75,-3.26,-2.55,-2.48,-2.25,-1.47,-1.13,-1.64,-2.04,-2.17,-2.81,-2.89,-3.54,-3.75,-3.92,-3.66,-3.09,-3.74,-5.90,-6.53,-7.82,-6.07,-5.95,-8.35,-7.72,-8.34,-9.31,-10.45,-8.17,-6.86,-5.71,-4.55,-4.01,-3.75,-3.08,-2.60,-1.44,-0.57,-0.31,-0.28,-1.65,-3.45
2015-01-01 04:00:00+00:00,-4.06,-5.44,-5.34,-5.53,-5.20,-6.02,-4.26,-3.92,-5.95,-4.37,-5.81,-5.50,-8.46,-9.63,-10.30,-8.88,-8.38,-8.39,-8.15,-6.25,-7.16,-8.48,-12.47,-11.25,-13.07,-12.77,-12.99,-13.17,-11.95,-9.47,-8.38,-9.12,-11.41,-10.81,-8.95,-11.49,-14.63,-14.96,-11.65,-10.83,-9.34,-10.69,-11.62,-9.75,-9.19,-6.96,-8.68,-8.38,-8.33,-7.08,...,-1.53,-1.62,-2.51,-3.60,-4.57,-5.99,-6.44,-5.59,-4.35,-4.85,-3.89,-3.99,-3.84,-3.03,-2.78,-3.19,-3.55,-3.66,-4.17,-4.41,-5.01,-5.20,-5.27,-4.83,-4.23,-5.01,-7.15,-8.02,-9.08,-7.33,-7.14,-9.15,-8.81,-9.06,-10.23,-11.33,-8.94,-7.52,-6.31,-5.19,-4.74,-4.62,-3.85,-3.31,-2.13,-1.27,-0.92,-0.87,-2.30,-3.92
...,...,...,...,...,

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 43824 entries, 2015-01-01 00:00:00+00:00 to 2019-12-31 23:00:00+00:00
Columns: 1260 entries, 975976 to 1038120
dtypes: float32(1260)
memory usage: 211.0 MB
meta_df:


,latitude,longitude,timezone,elevation,country,state,county,offshore,eez
gid,,,,,,,,,
975976,39.312191,-107.184372,-7.0,2344,United States,Colorado,Pitkin,0,1
975977,39.312191,-107.156250,-7.0,2507,United States,Colorado,Pitkin,0,1
975978,39.312191,-107.128128,-7.0,2474,United States,Colorado,Pitkin,0,1
975979,39.312191,-107.099998,-7.0,2481,United States,Colorado,Pitkin,0,1
975980,39.312191,-107.071877,-7.0,2440,United States,Colorado,Pitkin,0,1
...,...,...,...,...,...,...,...,...,...
1038116,38.554298,-106.059372,-7.0,2197,United States,Colorado,Chaffee,0,1
1038117,38.554298,-106.031250,-7.0,2188,United States,Colorado,Chaffee,0,1
1038118,38.554298,-106.003128,-7.0,2188,United States,Colorado,Chaffee,0,1


<class 'pandas.core.frame.DataFrame'>
Index: 1260 entries, 975976 to 1038120
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   latitude   1260 non-null   float32
 1   longitude  1260 non-null   float32
 2   timezone   1260 non-null   float32
 3   elevation  1260 non-null   int16  
 4   country    1260 non-null   object 
 5   state      1260 non-null   object 
 6   county     1260 non-null   object 
 7   offshore   1260 non-null   int16  
 8   eez        1260 non-null   int16  
dtypes: float32(3), int16(3), object(3)
memory usage: 61.5+ KB


### write/read to .h5 files locally for later use/viz
* use .h5 for large amounts of data (much faster read/write than .csv)


In [79]:
### save as .h5

### data itself
filename = "wind_data.h5"
df.to_hdf(filename, "MW", mode="w")

### meta for data above
filename = "wind_meta.h5"
meta_df.loc[df.columns].to_hdf(filename, "MW", mode="w")

In [80]:
### read back saved .h5 files

### data itself
filename = "wind_data.h5"
df = pd.read_hdf(filename)

### meta data
filename = "wind_meta.h5"
meta_df = pd.read_hdf(filename)

print("df:")
df
df.info()
print("meta_df:")
meta_df
meta_df.info()

df:


gid,975976,975977,975978,975979,975980,975981,975982,975983,975984,975985,975986,975987,975988,975989,975990,975991,975992,975993,975994,975995,975996,975997,975998,975999,976000,976001,976002,976003,976004,976005,976006,976007,976008,976009,976010,976011,976012,976013,976014,976015,976016,976017,976018,976019,976020,978276,978277,978278,978279,978280,...,1035816,1035817,1035818,1035819,1035820,1038076,1038077,1038078,1038079,1038080,1038081,1038082,1038083,1038084,1038085,1038086,1038087,1038088,1038089,1038090,1038091,1038092,1038093,1038094,1038095,1038096,1038097,1038098,1038099,1038100,1038101,1038102,1038103,1038104,1038105,1038106,1038107,1038108,1038109,1038110,1038111,1038112,1038113,1038114,1038115,1038116,1038117,1038118,1038119,1038120
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00+00:00,1.91,0.80,0.60,0.61,0.95,0.07,2.37,2.72,0.34,2.43,0.14,0.98,-2.12,-2.95,-3.89,-2.48,-1.88,-2.43,-1.98,0.16,-0.67,-2.13,-6.07,-4.17,-6.75,-5.73,-5.93,-6.27,-4.97,-2.41,-1.61,-2.52,-4.80,-4.11,-2.05,-5.15,-8.26,-8.34,-4.99,-4.69,-2.30,-4.39,-5.47,-3.62,-3.33,-0.66,-2.58,-2.45,-2.06,-0.68,...,2.28,2.20,1.18,0.27,-0.38,0.19,-0.36,0.70,1.81,1.40,2.09,2.27,2.28,3.13,3.34,2.96,2.36,2.32,1.80,1.71,0.87,0.87,0.68,1.11,1.63,0.98,-1.16,-1.89,-3.02,-1.07,-1.58,-3.58,-2.99,-3.15,-4.38,-5.43,-2.90,-1.83,-0.61,0.37,0.77,0.35,0.79,0.86,2.07,2.89,2.94,2.93,1.38,-0.25
2015-01-01 01:00:00+00:00,0.20,-1.20,-1.11,-1.20,-0.58,-1.52,0.55,0.95,-1.37,0.72,-1.20,-0.64,-3.72,-4.36,-5.41,-3.64,-3.46,-3.49,-3.47,-1.12,-2.27,-3.10,-7.70,-5.67,-8.28,-7.64,-7.49,-7.71,-6.64,-3.92,-3.05,-3.85,-6.15,-5.84,-4.23,-6.59,-9.57,-9.85,-6.82,-6.40,-4.11,-5.84,-6.79,-4.94,-4.65,-2.65,-4.72,-4.26,-3.73,-2.22,...,1.02,1.02,0.13,-1.03,-1.79,-0.84,-1.55,-0.45,0.67,0.18,0.99,1.02,1.04,1.72,2.14,1.69,1.32,1.14,0.37,0.27,-0.44,-0.63,-0.82,-0.35,0.15,-0.43,-2.89,-3.48,-4.97,-3.31,-2.86,-5.25,-4.75,-5.01,-5.87,-7.21,-4.94,-3.55,-2.40,-1.46,-1.06,-1.07,-0.75,-0.52,0.66,1.57,1.71,1.77,0.44,-1.47
2015-01-01 02:00:00+00:00,-2.09,-3.33,-3.16,-3.37,-2.98,-3.77,-1.85,-1.81,-3.64,-2.17,-3.29,-2.83,-6.07,-6.94,-7.64,-5.94,-5.98,-5.82,-5.60,-3.43,-4.65,-5.78,-9.84,-8.50,-10.86,-9.96,-10.07,-10.56,-9.17,-6.64,-5.52,-6.37,-8.62,-8.12,-6.56,-9.05,-11.97,-12.29,-9.40,-8.48,-6.71,-8.33,-9.15,-7.46,-6.97,-4.91,-6.88,-6.37,-5.98,-4.70,...,-0.09,-0.32,-1.10,-2.24,-3.00,-3.12,-3.72,-2.66,-1.77,-2.09,-1.12,-1.26,-1.09,-0.26,-0.01,-0.56,-0.85,-1.07,-1.76,-1.94,-2.46,-2.70,-2.89,-2.53,-1.96,-2.75,-5.03,-5.47,-6.88,-4.95,-5.14,-7.58,-6.96,-7.27,-8.31,-9.40,-6.98,-5.72,-4.41,-3.48,-3.05,-2.95,-2.35,-2.01,-0.76,0.18,0.47,0.48,-0.86,-2.60
2015-01-01 03:00:00+00:00,-3.27,-4.76,-4.40,-4.55,-4.13,-4.98,-3.23,-2.99,-4.75,-3.24,-4.44,-4.13,-7.12,-8.28,-9.14,-7.39,-7.16,-7.18,-7.12,-4.90,-6.10,-7.07,-11.01,-9.87,-11.87,-11.37,-11.36,-11.66,-10.59,-7.93,-6.92,-7.72,-9.95,-9.41,-7.92,-10.26,-13.42,-13.36,-10.44,-9.67,-7.88,-9.59,-10.29,-8.53,-8.16,-6.00,-8.17,-7.62,-7.28,-5.78,...,-0.98,-0.93,-1.84,-3.04,-3.99,-4.41,-5.03,-3.95,-2.75,-3.26,-2.55,-2.48,-2.25,-1.47,-1.13,-1.64,-2.04,-2.17,-2.81,-2.89,-3.54,-3.75,-3.92,-3.66,-3.09,-3.74,-5.90,-6.53,-7.82,-6.07,-5.95,-8.35,-7.72,-8.34,-9.31,-10.45,-8.17,-6.86,-5.71,-4.55,-4.01,-3.75,-3.08,-2.60,-1.44,-0.57,-0.31,-0.28,-1.65,-3.45
2015-01-01 04:00:00+00:00,-4.06,-5.44,-5.34,-5.53,-5.20,-6.02,-4.26,-3.92,-5.95,-4.37,-5.81,-5.50,-8.46,-9.63,-10.30,-8.88,-8.38,-8.39,-8.15,-6.25,-7.16,-8.48,-12.47,-11.25,-13.07,-12.77,-12.99,-13.17,-11.95,-9.47,-8.38,-9.12,-11.41,-10.81,-8.95,-11.49,-14.63,-14.96,-11.65,-10.83,-9.34,-10.69,-11.62,-9.75,-9.19,-6.96,-8.68,-8.38,-8.33,-7.08,...,-1.53,-1.62,-2.51,-3.60,-4.57,-5.99,-6.44,-5.59,-4.35,-4.85,-3.89,-3.99,-3.84,-3.03,-2.78,-3.19,-3.55,-3.66,-4.17,-4.41,-5.01,-5.20,-5.27,-4.83,-4.23,-5.01,-7.15,-8.02,-9.08,-7.33,-7.14,-9.15,-8.81,-9.06,-10.23,-11.33,-8.94,-7.52,-6.31,-5.19,-4.74,-4.62,-3.85,-3.31,-2.13,-1.27,-0.92,-0.87,-2.30,-3.92
...,...,...,...,...,

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 43824 entries, 2015-01-01 00:00:00+00:00 to 2019-12-31 23:00:00+00:00
Columns: 1260 entries, 975976 to 1038120
dtypes: float32(1260)
memory usage: 211.0 MB
meta_df:


,latitude,longitude,timezone,elevation,country,state,county,offshore,eez
gid,,,,,,,,,
975976,39.312191,-107.184372,-7.0,2344,United States,Colorado,Pitkin,0,1
975977,39.312191,-107.156250,-7.0,2507,United States,Colorado,Pitkin,0,1
975978,39.312191,-107.128128,-7.0,2474,United States,Colorado,Pitkin,0,1
975979,39.312191,-107.099998,-7.0,2481,United States,Colorado,Pitkin,0,1
975980,39.312191,-107.071877,-7.0,2440,United States,Colorado,Pitkin,0,1
...,...,...,...,...,...,...,...,...,...
1038116,38.554298,-106.059372,-7.0,2197,United States,Colorado,Chaffee,0,1
1038117,38.554298,-106.031250,-7.0,2188,United States,Colorado,Chaffee,0,1
1038118,38.554298,-106.003128,-7.0,2188,United States,Colorado,Chaffee,0,1


<class 'pandas.core.frame.DataFrame'>
Index: 1260 entries, 975976 to 1038120
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   latitude   1260 non-null   float32
 1   longitude  1260 non-null   float32
 2   timezone   1260 non-null   float32
 3   elevation  1260 non-null   int16  
 4   country    1260 non-null   object 
 5   state      1260 non-null   object 
 6   county     1260 non-null   object 
 7   offshore   1260 non-null   int16  
 8   eez        1260 non-null   int16  
dtypes: float32(3), int16(3), object(3)
memory usage: 61.5+ KB


### reading raw data via h5pyd
* possible, but need to download sup3rcc files that are about ~500GB for one year.

In [61]:
!hsstat /nrel/sup3rcc/conus_ecearth3_ssp585_r1i1p1f1/sup3rcc_conus_ecearth3_ssp585_r1i1p1f1_2053.h5

domain: /nrel/sup3rcc/conus_ecearth3_ssp585_r1i1p1f1/sup3rcc_conus_ecearth3_ssp585_r1i1p1f1_2053.h5
    owner:           nrel_admin
    id:              g-805bcf37-1e7135f2-08d3-47bf96-f9bd7a
    last modified:   2024-02-28 15:06:30
    last scan:       2024-02-28 14:40:48
    md5 sum:         14abbb672b058cc1f828723ea30a4cb7
    total_size:      552208282025
    allocated_bytes: 3312000
    metadata_bytes:  47676
    linked_bytes:    552204921848
    num objects:     27
    num chunks:      12
    linked chunks:   276065


In [78]:
# ### download one resource file (domain)
# ### would take about 70 hours (via my fast internet connection albeit with vnp)
# !hsget /nrel/sup3rcc/conus_ecearth3_ssp585_r1i1p1f1/sup3rcc_conus_ecearth3_ssp585_r1i1p1f1_2053.h5

### now can proceed to viz notebook